# Run First

In [1]:
from IPython.display import Markdown

def display_md(content):
  display(Markdown(content))

# Naive Indexer Architecture
In this course, we are indexing a small number of HTML files. However, in production situations, you're likely to encounter some situations that significantly increase the complexity of accurate indexing:  
- PDFs contain much of the world's unstructured data
- Parsing PDFs with vision can require layout understanding, which is not a generally solved problem
- PDFs often contain tables, graphics, footnotes, equations, etc that require special handling
- Many business cases require indexing highly heterogenous document layouts

Since accurately indexing files is the beginning of your inference pipeline, this is often one of the most consequential engineering problems to perform well at.

## loaders.py
Look at `./workshop-code/indexer_components/loaders.py`

The function here is pretty simple. It downloads a file at a specified URI, then saves it to a cache so it doesn't need to be downloaded on each subsequent run in the notebook. The preprocessor will consume this file in the next step. 

### Task: Read the Code
This part isn't interesting, so just look at the code and understand what it does. If you have any questions, let one of us know.

Note that the proper way to do caching is by using the HTTP response's `ETag`, `Last-Modified`, and `Cache-Control` headers, but I didn't do that here. If you want extra credit, feel free to send me a pull request with the corrected code.

## preprocessors.py
Look at `./workshop-code/indexer_components/preprocessors.py`

In order to reduce inference costs, we want to strip away all of the HTML syntax besides the human-readable body text of the documents. This is a design decision. In real life, you may, for example, preserve more of the HTML markup to reain the context of how the document is structured.

### Easy Task: Configure Beautiful Soup for A Simply Structured Blog Post

The unprocessed HTML blog post looks like this:

In [2]:
from workshop_code.indexer_components.loaders       import DocLoader

blog_post_uri = "https://lilianweng.github.io/posts/2023-06-23-agent/"
doc_content = DocLoader.load_html(blog_post_uri)
display_md(doc_content[3400:3800])

/Users/jacob/Development/rag-mini-bootcamp/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


tle" content="LLM Powered Autonomous Agents" />
<meta property="og:description" content="Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be frame

The processed HTML blog post should like like this:

In [3]:
from cheat_code.indexer_components.loaders       import DocLoader
from cheat_code.indexer_components.preprocessors import GithubBlogpostPreprocessor

blog_post_uri = "https://lilianweng.github.io/posts/2023-06-23-agent/"
blog_post_html = DocLoader.load_html(blog_post_uri)
preprocessor = GithubBlogpostPreprocessor()
cleaned_doc_content = preprocessor.get_text(blog_post_html)

display_md(cleaned_doc_content[0:1000])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

Complete the method `GithubBlogpostPreprocessor.get_text()` in `./workshop_code/preprocessors.py` such that the `cleaned_text` looks like the above output.

In [4]:
from workshop_code.indexer_components.loaders       import DocLoader
from workshop_code.indexer_components.preprocessors import GithubBlogpostPreprocessor

blog_post_uri = "https://lilianweng.github.io/posts/2023-06-23-agent/"
blog_post_html = DocLoader.load_html(blog_post_uri)
preprocessor = GithubBlogpostPreprocessor()
cleaned_doc_content = preprocessor.get_text(blog_post_html)

display_md(cleaned_doc_content[0:1000])


      LLM Powered Autonomous Agents
    

## Medium Task: Copy or Write a Preprocessor for the RAG Survey Paper
The HTML structure of the Arxiv paper is more complex than the blog post. You can try implementing some of `ArxivHtmlPaperPreprocessor` to see for yourself. But, I suggest just copying the cheat-code. 

Here is the working implementation:

In [5]:
from cheat_code.indexer_components.loaders       import DocLoader
from cheat_code.indexer_components.preprocessors import ArxivHtmlPaperPreprocessor

rag_survey_paper_uri = "https://arxiv.org/html/2312.10997v5"
rag_survey_paper_html = DocLoader.load_html(rag_survey_paper_uri)
preprocessor = ArxivHtmlPaperPreprocessor()
cleaned_doc_content = preprocessor.get_text(rag_survey_paper_html)

display_md(cleaned_doc_content[0:2000])

Retrieval-Augmented Generation for Large Language Models: A Survey

Yunfan Gao: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University


Yun Xiong: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Xinyu Gao: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Kangxiang Jia: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Jinliu Pan: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Yuxi Bi: College of Design and Innovation, Tongji University


Yi Dai: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University


Jiawei Sun: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University


Meng Wang: College of Design and Innovation, Tongji University


Haofen Wang: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University College of Design and Innovation, Tongji University



Abstract

Large Language Models (LLMs) showcase impressive capabilities but encounter challenges like hallucination, outdated knowledge, and non-transparent, untraceable reasoning processes. Retrieval-Augmented Generation (RAG) has emerged as a promising solution by incorporating knowledge from external databases. This enhances the accuracy and credibility of the generation, particularly for knowledge-intensive tasks, and allows for continuous knowledge updates and integration of domain-specific information. RAG synergistically merges LLMs’ intrinsic knowledge with the vast, dynamic repositories of external databases. This comprehensive review paper offers a detailed examination of the progression of RAG paradigms, encompassing the Naive RAG, the Advanced RAG, and the Modular RAG. It meticulously scrutinizes the tripartite foundation of RAG frameworks, which includes the retrieval, the generation and the augmentation techniques. The paper highlights the s

And, here is the implementation for you to complete:

In [6]:
from workshop_code.indexer_components.loaders       import DocLoader
from workshop_code.indexer_components.preprocessors import ArxivHtmlPaperPreprocessor

rag_survey_paper_uri = "https://arxiv.org/html/2312.10997v5"
rag_survey_paper_html = DocLoader.load_html(rag_survey_paper_uri)
preprocessor = ArxivHtmlPaperPreprocessor()
cleaned_doc_content = preprocessor.get_text(rag_survey_paper_html)

display_md(cleaned_doc_content[0:2000])

Retrieval-Augmented Generation for Large Language Models: A Survey

Yunfan Gao: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University


Yun Xiong: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Xinyu Gao: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Kangxiang Jia: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Jinliu Pan: Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University


Yuxi Bi: College of Design and Innovation, Tongji University


Yi Dai: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University


Jiawei Sun: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University


Meng Wang: College of Design and Innovation, Tongji University


Haofen Wang: Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University College of Design and Innovation, Tongji University



Abstract

Large Language Models (LLMs) showcase impressive capabilities but encounter challenges like hallucination, outdated knowledge, and non-transparent, untraceable reasoning processes. Retrieval-Augmented Generation (RAG) has emerged as a promising solution by incorporating knowledge from external databases. This enhances the accuracy and credibility of the generation, particularly for knowledge-intensive tasks, and allows for continuous knowledge updates and integration of domain-specific information. RAG synergistically merges LLMs’ intrinsic knowledge with the vast, dynamic repositories of external databases. This comprehensive review paper offers a detailed examination of the progression of RAG paradigms, encompassing the Naive RAG, the Advanced RAG, and the Modular RAG. It meticulously scrutinizes the tripartite foundation of RAG frameworks, which includes the retrieval, the generation and the augmentation techniques. The paper highlights the s

## Hard Task, not recommended for today: Write a Preprocessor for the PDF Version of the RAG Survey Paper
In production applications, you're likely to need to do inference on PDFs. Today this is often a non-trivial task. The most popular open source solution is Tesseract. However, Tesseract often underperforms computer vision-based services from vendors like Google Cloud and AWS.

## text_splitters.py
Because LLM context windows are limited, semantic indexing strategies rely on text splitting. In this tutorial, we use the most naive strategy, character text splitting. To find inspiration or source code for more strategies, I look at LlamaIndex and Langchain. However, in some production situations, it will make sense to write a text splitter specific to your needs.

### Text splitting task #1: examine and copy the code for the text splitter
Here is the working implementation:

In [7]:
from cheat_code.indexer_components.loaders        import DocLoader
from cheat_code.indexer_components.preprocessors  import ArxivHtmlPaperPreprocessor
from cheat_code.indexer_components.text_splitters import SimpleCharacterTextSplitter

CHUNK_SIZE = 250
OVERLAP_SIZE = 25
rag_survey_paper_uri = "https://arxiv.org/html/2312.10997v5"
preprocessor = ArxivHtmlPaperPreprocessor()
text_splitter = SimpleCharacterTextSplitter(CHUNK_SIZE, OVERLAP_SIZE)

rag_survey_paper_html = DocLoader.load_html(rag_survey_paper_uri)
cleaned_doc_content = preprocessor.get_text(rag_survey_paper_html)
text_splits = text_splitter.split_text(cleaned_doc_content)

display_md(text_splits[3])

approaches in their respective contexts, and speculate on upcoming trends and innovations. Our contributions are as follows: In this survey, we present a thorough and systematic review of the state-of-the-art RAG methods, delineating its evolution through paradigms including naive RAG, advanced RAG, and modular RAG. This review contextualizes the broader scope of RAG research within the landscape of LLMs. We identify and discuss the central technologies integral to the RAG process, specifically focusing on the aspects of “Retrieval”, “Generation” and “Augmentation”, and delve into their synergies, elucidating how these components intricately collaborate to form a cohesive and effective RAG framework. We have summarized the current assessment methods of RAG, covering 26 tasks, nearly 50 datasets, outlining the evaluation objectives and metrics, as well as the current evaluation benchmarks and tools. Additionally, we anticipate future directions for RAG, emphasizing potential enhancements to tackle current challenges. The paper unfolds as follows: SectionIIintroduces the main concept and current paradigms of RAG. The following three sections explore core components—“Retrieval”, “Generation” and “Augmentation”, respectively. SectionIIIfocuses on optimization methods in retrieval,including indexing, query and embedding optimization. SectionIVconcentrates on post-retrieval process and LLM fine-tuning in generation. SectionVanalyzes the three augmentation processes. SectionVIfocuses on RAG’s downstream tasks and evaluation system. SectionVIImainly discusses the challenges that RAG currently faces and its future development directions. At last, the paper concludes in SectionVIII. IIOverview of RAG A typical application of RAG is illustrated in Figure2. Here, a user poses a question to ChatGPT about a recent, widely discussed news. Given ChatGPT’s reliance on pre-training data, it initially lacks the capacity to provide updates on recent developments. RAG bridges this information gap by sourcing and incorporating knowledge

Copy the code from `cheat_code/indexer_components/text_splitters.py` to `workshop_code/` so that the code below works: 

In [8]:
from workshop_code.indexer_components.loaders        import DocLoader
from workshop_code.indexer_components.preprocessors  import ArxivHtmlPaperPreprocessor
from workshop_code.indexer_components.text_splitters import SimpleCharacterTextSplitter

CHUNK_SIZE = 250
OVERLAP_SIZE = 25
rag_survey_paper_uri = "https://arxiv.org/html/2312.10997v5"
preprocessor = ArxivHtmlPaperPreprocessor()
text_splitter = SimpleCharacterTextSplitter(CHUNK_SIZE, OVERLAP_SIZE)

rag_survey_paper_html = DocLoader.load_html(rag_survey_paper_uri)
cleaned_doc_content = preprocessor.get_text(rag_survey_paper_html)
text_splits = text_splitter.split_text(cleaned_doc_content)

display_md(text_splits[3])

approaches in their respective contexts, and speculate on upcoming trends and innovations. Our contributions are as follows: In this survey, we present a thorough and systematic review of the state-of-the-art RAG methods, delineating its evolution through paradigms including naive RAG, advanced RAG, and modular RAG. This review contextualizes the broader scope of RAG research within the landscape of LLMs. We identify and discuss the central technologies integral to the RAG process, specifically focusing on the aspects of “Retrieval”, “Generation” and “Augmentation”, and delve into their synergies, elucidating how these components intricately collaborate to form a cohesive and effective RAG framework. We have summarized the current assessment methods of RAG, covering 26 tasks, nearly 50 datasets, outlining the evaluation objectives and metrics, as well as the current evaluation benchmarks and tools. Additionally, we anticipate future directions for RAG, emphasizing potential enhancements to tackle current challenges. The paper unfolds as follows: SectionIIintroduces the main concept and current paradigms of RAG. The following three sections explore core components—“Retrieval”, “Generation” and “Augmentation”, respectively. SectionIIIfocuses on optimization methods in retrieval,including indexing, query and embedding optimization. SectionIVconcentrates on post-retrieval process and LLM fine-tuning in generation. SectionVanalyzes the three augmentation processes. SectionVIfocuses on RAG’s downstream tasks and evaluation system. SectionVIImainly discusses the challenges that RAG currently faces and its future development directions. At last, the paper concludes in SectionVIII. IIOverview of RAG A typical application of RAG is illustrated in Figure2. Here, a user poses a question to ChatGPT about a recent, widely discussed news. Given ChatGPT’s reliance on pre-training data, it initially lacks the capacity to provide updates on recent developments. RAG bridges this information gap by sourcing and incorporating knowledge

### Text Splitting Task #2: look at alternative text splitters
Make a mental note of the other text splitters available here:
- [Langchain: Text Splitters](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)
- [LlamaIndex: Text Splitters](https://medium.com/@bavalpreetsinghh/llamaindex-chunking-strategies-for-large-language-models-part-1-ded1218cfd30)

## Embeddings: vectorizers.py
Embeddings of text is currently the most common method of preparing human-readable text so that they can be compared to each other for relatedness. Currently, OpenAI's embedding models rank amongst the highest performing, so we use theirs.  

OpenAI's text embedding models take up to 8191 tokens as input and convert them to a vector of dimension 1536 for `text-embedding-3-small` or 3072 for `text-embedding-3-large`.

### Embeddings Task #1: Use OpenAI's Embeddings API
The embeddings code for your naive RAG pipeline should behave like this:

In [9]:
from cheat_code.common_components.vectorizers import Vectorizer

example_text_splits = ["Mary had a", "little lamb"]
vectorizer = Vectorizer()
embeddings_of_example_splits = vectorizer.vectorize_text_splits(example_text_splits)

rows = len(embeddings_of_example_splits)
columns = len(embeddings_of_example_splits[0])

print(f"Dimensions: {rows}x{columns}")
print(embeddings_of_example_splits[0:10])

Dimensions: 2x1536
[[0.022420611, 0.01908309, -0.022541732, 0.04556794, -0.040561657, -0.011593852, 0.023026211, 0.017912265, -0.003990223, -0.025394775, 0.038542997, 0.0031995801, 0.03248701, 0.056522552, 0.008478383, 0.005574873, 0.03375204, -0.015772482, -0.01184282, 0.0392428, 0.0487978, 0.024048999, 0.048770886, -0.014278673, 0.016741442, -0.0180603, 0.005097123, -0.04443749, 0.017966097, 0.007556527, 0.03978111, -0.027359607, 0.048071083, -0.0092252875, -0.0047775013, 0.045944758, 0.037412547, 0.017024053, -0.0015947435, 0.010557605, -0.012616641, 0.016122384, 0.0430648, -0.0072941007, 0.0068567237, -0.0034031286, -0.037735533, 0.036766574, 0.0592141, 0.03606677, -0.009131083, 0.010073126, -0.0023315551, 0.025435148, -0.006537102, -0.04976676, -0.02021354, 0.0028715474, 0.033402137, -0.019163836, -0.025381317, -0.0073748473, 0.022743598, -0.032513924, -0.007442136, -0.027548015, 0.02524674, 0.040776983, -0.035313137, -0.0068735457, 0.05407324, 0.03267542, -0.0090570655, 0.0349094

Implement `vectorize_text_splits()` in `workshop_code/common_components/vectorizers.py` by referencing the [OpenAI embedding API's documentation](https://platform.openai.com/docs/api-reference/embeddings).

In [10]:
from workshop_code.common_components.vectorizers import Vectorizer

example_text_splits = ["Mary had a", "little lamb"]
vectorizer = Vectorizer()
embeddings_of_example_splits = vectorizer.vectorize_text_splits(example_text_splits)

rows = len(embeddings_of_example_splits)
columns = len(embeddings_of_example_splits[0])

print(f"Dimensions: {rows}x{columns}")
print(embeddings_of_example_splits[0:10])

Dimensions: 2x1536
[[0.022420611, 0.01908309, -0.022541732, 0.04556794, -0.040561657, -0.011593852, 0.023026211, 0.017912265, -0.003990223, -0.025394775, 0.038542997, 0.0031995801, 0.03248701, 0.056522552, 0.008478383, 0.005574873, 0.03375204, -0.015772482, -0.01184282, 0.0392428, 0.0487978, 0.024048999, 0.048770886, -0.014278673, 0.016741442, -0.0180603, 0.005097123, -0.04443749, 0.017966097, 0.007556527, 0.03978111, -0.027359607, 0.048071083, -0.0092252875, -0.0047775013, 0.045944758, 0.037412547, 0.017024053, -0.0015947435, 0.010557605, -0.012616641, 0.016122384, 0.0430648, -0.0072941007, 0.0068567237, -0.0034031286, -0.037735533, 0.036766574, 0.0592141, 0.03606677, -0.009131083, 0.010073126, -0.0023315551, 0.025435148, -0.006537102, -0.04976676, -0.02021354, 0.0028715474, 0.033402137, -0.019163836, -0.025381317, -0.0073748473, 0.022743598, -0.032513924, -0.007442136, -0.027548015, 0.02524674, 0.040776983, -0.035313137, -0.0068735457, 0.05407324, 0.03267542, -0.0090570655, 0.0349094

## Vector Database: wcs_client_adapter.py
In production settings, you're likely to store your vectors in a database. In this tutorial, we are using Weaviate, which is abstracted away by `wcs_client_adapter.py`. However, there are many options available. Pinecone has been the most popular startup vector database provider, but popular existing players such as Postgres also offer vector storage.

### Vector DB Task #1: Understand the indexer code
Open `indexers.py` in `./workshop_code/`. Look over how the `WcsClientAdapter` is used, and look at how its methods are implemented. If something doesn't make sense, ask a question.

## The Complete Indexer: indexer.py
### Indexer Task: test that your indexer works
Your indexer should give output like the cheat_code version below:

In [11]:
from cheat_code.indexers import NaiveIndexer
from cheat_code.common_components.vectorizers import Vectorizer

rag_survey_paper_uri = "https://arxiv.org/html/2312.10997v5"
vectorizer = Vectorizer()
indexer = NaiveIndexer(vectorizer)
indexer.index(rag_survey_paper_uri)
num_db_entries = indexer._wcs_client_adapter.count_entries()
print(f"Number of text chunks in Weaviate: {num_db_entries}")

E0526 13:08:33.998605000 8307980992 ssl_transport_security.cc:1654]    Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.
E0526 13:08:34.018567000 8307980992 ssl_transport_security.cc:1654]    Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.


WeaviateGRPCUnavailableError: 
Weaviate v1.24.14 makes use of a high-speed gRPC API as well as a REST API.
Unfortunately, the gRPC health check against Weaviate could not be completed.

This error could be due to one of several reasons:
- The gRPC traffic at the specified port is blocked by a firewall.
- gRPC is not enabled or incorrectly configured on the server or the client.
    - Please check that the server address and port (rag-mini-lp1ht9yf.grpc.weaviate.network:443) are correct.
- your connection is unstable or has a high latency. In this case you can:
    - increase init-timeout in `weaviate.connect_to_local(additional_config=wvc.init.AdditionalConfig(timeout=wvc.init.Timeout(init=X)))`
    - disable startup checks by connecting using `skip_init_checks=True`


Run your indexer below to see if it works the same way. If it doesn't, something is broken.

In [ ]:
from workshop_code.indexers import NaiveIndexer
from workshop_code.common_components.vectorizers import Vectorizer

rag_survey_paper_uri = "https://arxiv.org/html/2312.10997v5"
vectorizer = Vectorizer()
indexer = NaiveIndexer(vectorizer)
indexer.index(rag_survey_paper_uri)
num_db_entries = indexer._wcs_client_adapter.count_entries()
print(f"Number of text chunks in Weaviate: {num_db_entries}")

ModuleNotFoundError: No module named 'indexer_components'